In [1]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [2]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [3]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [4]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [5]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [6]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
26852,Tres extranjeros incumplían la cuarentena tota...,@usuario Fuera de aca
10525,Video: el momento en que un policía mata a bal...,@usuario Que lindo ver como mueren estas ratas...


In [7]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [8]:
from pprint import pprint as pp
from rioplatense_hs.prompting import build_prompt

print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: Mujeres o movimiento feminista
- lgbti: Género, identidad sexual
- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
- clase: Clase, situación socioeconómica o barrio de residencia
- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas
- política: Opinión o ideología política.
- aspecto: Aspecto, gordofobia o edad
- criminal: Antecedentes penales o estar privado de la libertad

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá paso

In [13]:
# sample_df = df_test.sample(100)

# sample_df = df_dev.sample(700, random_state=2022)

sample_df = pd.read_csv("../data/sample_100_pred.csv")

In [14]:
(sample_df["num_hatred"] > 0).mean()

0.18

In [19]:
sample_df["prompt"]

0     Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:\n\n- mujer: Mujeres o movimiento feminista\n- lgbti: Género, identidad sexual\n- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades\n- clase: Clase, situación socioeconómica o barrio de residencia\n- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas\n- política: Opinión o ideología política.\n- aspecto: Aspecto, gordofobia o edad\n- criminal: Antecedentes penales o estar privado de la libertad\n\nResponder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discrimina

In [11]:
outs = {

}

In [13]:
sample_df.iloc[:100].to_csv("../data/sample_100.csv", index=False)

In [163]:
from tqdm import tqdm
import time
from rioplatense_hs.prompting import get_response


for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    prompt, response = get_response(contexto=row["context_tweet"], texto=row["text"])

    outs[idx] = (prompt, response)

    time.sleep(0.01)

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 700/700 [18:21<00:00,  1.57s/it]


In [164]:
sample_df["pred_cot"] = sample_df.index.map(lambda x: outs[x][1])
sample_df["prompt"] = sample_df.index.map(lambda x: outs[x][0])


In [165]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
442,"Sentidas palabras de Morena Rial: “Ahora sonrío al ver una foto mía en lugar de llorar, luché para aceptarme” https://t.co/dRCmt6Ahlk",@usuario El problema es q ahora tiene la cara de guido suller y ahi no hay marcha atras mamu eh.,nada,"El texto alude al aspecto de Morena Rial y compara su cara con la de Guido Suller de manera despectiva. La respuesta final es ""aspecto"""
535,El Gobierno analiza mantener el bono de $10.000 para los sectores más vulnerables después de la pandemia https://t.co/JvmcFDuAaA,@usuario Está bien sigan fomentando vagos.,clase,"El comentario se refiere a quienes reciben el bono de $10.000 como ""vagos"". La respuesta final es ""clase"""
301,Organizaba marchas anti cuarentena y murió por coronavirus: “Mi primo decía que nadie lo iba a frenar” https://t.co/WAaCSNnlVm,"@usuario La mayoria de las personas mayores se manejan asi, son terribles faltadores de respeto y viven sin importarles el resto, egoismo puro",nada,"El texto alude a las personas mayores como ""terribles faltadores de respeto"" y egoístas. La respuesta final es ""edad"""
529,🔴 AHORA | Alberto Fernández le contestó a Macri: “La primera condición para ser libres es que estemos vivos” https://t.co/jLNHx1u2RU https://t.co/tSRXGleKkZ,@usuario A Alberto le falta decir que es dos vidas..... Jajajajajaajajajajajajaja,nada,"El texto se refiere a Alberto Fernández y menciona su posición política, pero no contiene declaraciones de rechazo, enemistad o aborrecimiento irracional. Por lo tanto, la respuesta final es ""política""."
185,Ximena Capristo publicó escandalosos chats de Gustavo Conti con otra mujer: “Uno se termina enterando de todo” https://t.co/lO9gkEswFX,@usuario Ximena con un odio del 87.5%\n\nMuy bien por ella.,nada,"El texto alude a la actitud de Ximena Capristo, describiéndola con odio. La respuesta final es ""nada"""


In [166]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)

## Error analysis

In [167]:
import pandas as pd

pd.options.display.max_colwidth = 1000

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [170]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("respuesta final")][["context_tweet", "text", "text_label","pred_cot"]]

,context_tweet,text,text_label,pred_cot
111,"Juana Repetto, indignada por su viaje a Miami: '¡El avión iba explotado! Era un hormiguero' https://t.co/4r85gxRz4g",@usuario #Quédateencasa Juani. Jajaja q pelotudos q son,nada,El texto no contiene discurso discriminatorio.
200,Condenaron a 10 años de prisión a “la Angelina Jolie iraní” https://t.co/eEjQwkhNuq,"@usuario Angelina Jolie, después de inyectarse la Sputnik V",nada,El mensaje no contiene discurso discriminatorio.
270,[OPINIÓN] Aborto legal: otra promesa incumplida | Por Romina Del Pla https://t.co/m1gIXwm9TO,"@usuario @usuario Me decepcionaste, decir que @usuario es culpable de la vuelta de los K, 4 años tuvo el inútil de @usuario, no conoces el liberalismo @usuario edúcate.",nada,El texto no contiene discurso discriminatorio.


In [171]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [180]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.31      0.43      0.36        21
       LGBTI       0.40      0.46      0.43        13
      RACISM       0.45      0.80      0.58        40
       CLASS       0.14      0.60      0.23        10
    POLITICS       0.15      0.77      0.26        22
    DISABLED       0.22      0.40      0.29         5
  APPEARANCE       0.27      0.64      0.38        22
    CRIMINAL       0.26      0.53      0.35        17

   micro avg       0.26      0.63      0.37       150
   macro avg       0.28      0.58      0.36       150
weighted avg       0.31      0.63      0.40       150
 samples avg       0.11      0.12      0.11       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [174]:
label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.94      0.65      0.77       582
        True       0.32      0.80      0.45       118

    accuracy                           0.68       700
   macro avg       0.63      0.72      0.61       700
weighted avg       0.84      0.68      0.72       700



In [175]:
df_train[df_train["POLITICS"] > 0 ]

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label
7,360180,Vicentin: balearon a un manifestante durante la protesta frente a la Quinta de Olivos,@usuario lastima que no murio... necesitamos unos cuantos martires para que los zurdos de mierda dejen de sobarle los huevos a la estatua de peron... sucios peronchitos... y los macristas de mierda no se quedan atras.,Vicentin: balearon a un manifestante durante la protesta frente a la Quinta de Olivos https://t.co/zm3Mj37oL0,1,"En un extraño episodio que todavía se investiga, un hombre fue baleado a metros de la quinta presidencial de Olivos, a donde había ido a apoyar una de las manifestaciones que se realizaron en todo el país en contra de la expropiación de Vicentin y en defensa de la propiedad privada.\n\nR.J.A., vecino de San Isidro, de 49 años, se acercó hasta a la residencia donde descansa Alberto Fernández acompañado de su esposa. Llegaron en su auto poco después de las 16:30, hora pactada para el inicio del banderazo que se repitió en casi 10 distritos del país. Para entonces, la policía ya había cortado el tránsito en cercanías a la quinta presidencial.\n\nLuego de estacionar su vehículo sobre la Avenida Maipú, en sentido capital y antes del cruce con la calle Acassuso, la pareja se bajó del rodado y empezó a caminar hacia al grueso de la convocatoria. Con su teléfono celular, R.J.A. comenzó tomar una fotos y filmar unos videos hasta que de repente sintió un fuerte impacto en su brazo derecho: h...",0,0,0,0,0,1,0,0,0,1,política
17,399158,Baby Etchecopar fue golpeado por motochorros y debió ser trasladado de urgencia a un hospital,"@usuario Justo cuando salio de su programa de radio, ¿¿podrían disimular un poco los kukas no?? Pero bueh como dijo Perón: ""al enemigo ni justicia” más fascista imposible.",Baby Etchecopar fue golpeado por motochorros en un asalto y debió ser trasladado de urgencia a un hospital https://t.co/qqF4iwoTC6,1,"Este jueves por la tarde Baby Etchechopar fue víctima de un episodio delictivo cuando dos motochorros lo abordaron de manera violenta al retirarse de Radio Rivadavia, ubicada en la calle Arenales al 2400, en Barrio Norte. En esa emisora conduce el programa Baby en el medio, de 12 a 14.\n\nEtchechopar caminaba por la vereda cuando fue sorprendido por los delincuentes. Hubo un fuerte forcejeo, ya que se resistió a que le quitaran su reloj. Pero no pudo evitar el robo y, como consecuencia de la disputa con los maleantes, se lastimó un hombro. Un agente de la Policía de la Ciudad vio el robo, detuvo a uno de los ladrones y recuperó el reloj. El otro se fugó en la moto rápidamente.\n\nEl también conductor de Basta Baby, ciclo que se emite por A24, fue llevado de inmediato al Hospital de San Isidro, ya que pidió ser atendido por su traumatólogo de confianza. Según las primeras informaciones, tuvo una luxación en el hombro, pero su cuadro no revestiría gravedad.\n\nEsta no es la primera v...",0,0,0,0,0,1,0,0,0,1,política
34,406104,A tres años de la masacre del Junquito: el día que las fuerzas de Maduro ejecutaron en vivo y directo al policía Óscar Pérez,"@usuario uno menos, este no jode mas.",[IMÁGENES SENSIBLES] A tres años de la masacre del Junquito: el día que las fuerzas de Maduro ejecutaron en vivo y directo al policía Óscar Pérez https://t.co/YbVnqycXA9,1,"(Carlos Crespo, La Gran Aldea)-""Dijimos que nos íbamos a entregar y no quieren dejar que nos entreguemos, nos quieren asesinar"", estas fueron las palabras pronunciadas en uno de los últimos videos que transmitió el ex policía Óscar Pérez desde sus redes sociales, poco antes de ser asesinado por cuerpos de seguridad el 15 de enero de 2018, junto a otras seis personas que le acompañaban, en lo que organizaciones como el Programa Venezolano de Educación-Acción en Derechos Humanos (Provea) calificaron como la ""masacre de El Junquito"".\n\nEl integrante de Provea, Marino Alvarado, indicó que esa organización regist

In [176]:
sample_df[pred_hate != label_hate][["context_tweet", "text", "text_label", "pred_cot", ]].sample(10)

,context_tweet,text,text_label,pred_cot
291,Preocupación por un posible 'efecto contagio' de coronavirus en la masiva despedida a Diego Maradona https://t.co/H2FLlC0khR,"@usuario Que se caguen muriendo, así bajan los planes",nada,"El texto muestra una actitud de desprecio hacia aquellos que reciben planes sociales, deseando que mueran. La respuesta final es ""clase""."
395,El video de la secuencia completa del robo al jubilado de Quilmes que mató a un ladrón https://t.co/QulKn9hPq8,@usuario A alguien le importo como el DELINCUENTE entro a propiedad ajena para atacar a un ciudadano?,nada,"El texto se refiere al ladrón como un delincuente y alude a su acción de entrar a una propiedad ajena para atacar a una persona. La respuesta final es ""criminal""."
128,"Coronavirus. Eugenio Zaffaroni defendió la liberación de presos: ""Nadie habla de soltar a violadores y homicidas, sino de evitar una masacre"" https://t.co/azchUiVvBt https://t.co/dLKfmfnEM7",@usuario Depende dnde sea la masacre. Presos o sueltos?,nada,"El texto pregunta si la masacre será de presos o de personas sueltas, lo cual implica cierta discriminación hacia los presos. La respuesta final es ""criminal""."
463,"Hebe de Bonafini le mandó una carta a la reina de Inglaterra: ""Su país decidió robarle los ahorros en oro a Venezuela"" https://t.co/VDVwKls6Os","@usuario @usuario avÍsele a la reina q esta terrorista, está totalmente loKA, oKuparon el país del cual somos rehenes mas del 52% de los argentinos,no estamos c/el diKtador Maduro, lo único q queremos es vivir en paz, sin terroristas q hablen x nosotros Thank you Mr Kent 🥰🤗",nada,"El texto alude a Hebe de Bonafini como una ""terrorista"" y ""loka"" (loca). Además, menciona al ""diKtador Maduro"" en relación con Venezuela, insinuando una conexión entre los argentinos y el terrorismo. La respuesta final es ""política""."
235,La científica argentina Sandra Pitta apuntó su visión crítica sobre el manejo de la pandemia: “Se hicieron las cosas mal en la Argentina” | Por Daniela Blanco https://t.co/j7IqUGoNEY,@usuario UNA POBRE MEDIOCRE CON DELIRIOS DE IMPORTANCIA USADA Y BURLADA POR LOS QUE ELLA APOYA!!!\nEL NIVEL DE DAÑO AUTOINFLINGIDO ES IGUAL A SU ODIO POR LOS QUE HACEN TODO BIEN Y SON SUPERIORES: ENVIDIA!!!,nada,"El texto insulta y descalifica a la científica (pobre mediocre, con delirios de importancia) y expresa odio hacia aquellos a quienes ella apoya. La respuesta final es ""política""."
488,Efecto coronavirus: ¿ahora la grieta es entre Alberto Fernández y Cristina Kirchner? https://t.co/wM6sIQRMZ6,@usuario CFK tiene coronita? volver de Cuba en momentos que nadie puede hacerlo!!! Estan diciendole a la gente que desde el momento que se decreto la cuarenta TODOS deben quedarse en el lugar que esten! Yo no pude volver a mi casa y esta señora vuelve como si nada y encima a su depto!!!,nada,"El texto alude a que Cristina Kirchner tiene privilegios para viajar y no cumplir con las normas de la cuarentena, en contraste con el autor del mensaje que no pudo regresar a su hogar. La respuesta final es ""política"""
508,"Tensión entre Frederic y Berni: ""¿Vas a desenfundar o me atacás por ser mujer? "" https://t.co/W0r71FIqiq https://t.co/f7fzFWW8Me",@usuario NI DESENFUNDAR...NI ATACAR...SUPER BERNI SÓLO DEBE SOPLAR FUERTE Y LA GENIA NO DESCUBIERTA SE CAE...,nada,"El texto se refiere a su condición de mujer de manera despectiva, pero no contiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento. La respuesta final es ""mujer""."
455,"Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: ""Aprendan de nuestros errores"" https://t.co/mBLG1XfOGD","@usuario Lo que tenemos que aprender es como iniciarles una demanda por daños y perjuicios, tienen que pagar por la cagada que se mandaron y la gente que se murió y se sigue muriendo",racismo,El texto no contiene discurso discriminatorio. La respuesta final es 'nada'.
23,"Coronavirus: ¿China logró la vacuna "

In [177]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
690,"En promedio, los beneficiarios de planes sociales cobrarán $40.000 en diciembre | Por Pablo Wende https://t.co/OaSD2ZmmHy",@usuario @usuario Les seguimos pagando el crédito de ANSES con el q se compraron el 50’ \nArroyo mismo lo dijo cuando empezó la cuareterna: -muchos tienen créditos y les llega descontado del plan”-\nPagamos sus vicios y sus “gustitos “,nada,"El texto se refiere a los beneficiarios de planes sociales como personas que gastan el dinero en vicios y gustitos. La respuesta final es ""clase"""
294,"Juan Grabois, sobre la post pandemia: “Si no hay una intervención política, va a haber un fuerte aumento de la criminalidad y un proceso de toma de tierras” https://t.co/h5wlZGPRQd",@usuario Dnd vive Grabois ?? En algún asentamiento ó barrio privado ??,nada,"El texto cuestiona el lugar de residencia de Juan Grabois, sugiriendo la posibilidad de que viva en un asentamiento o barrio privado. La respuesta final es ""clase"""
682,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Espero que la gente, dueña,.los saque a patadas en el culo. Quien se creen que son. Tomar tierra, trabaja y pedí un crédito. Por Asís e hacen las cosas, así hacen el resto de los argentinos que trabaja.","clase, criminal",El texto no contiene discurso discriminatorio. La respuesta final es 'nada'
149,El Polo Obrero realiza un corte en la Panamericana en contra de la flexibilización de la cuarentena y en reclamo de aumentos a los planes sociales https://t.co/fHOeCKJdY6,"@usuario A ver si entendí, cortan para q nadie vaya a laburar y q leaumente la limosna ??!!!",nada,"El texto critica a los manifestantes por cortar la Panamericana y reclamar aumentos a los planes sociales, llamándolos ""limosna"". La respuesta final es ""clase"""
93,Pablo Moyano amenaza con un paro de recolectores en la Ciudad de Buenos Aires https://t.co/tPArqK6otB https://t.co/8F6nHRi0Jr,@usuario CUANTA PLATA TIENEN ENTRE ESTOS TRES TIPOS. SON MULTIMILLONARIOS.,nada,"El texto se refiere a la cantidad de dinero que tienen entre los tres mencionados. La respuesta final es ""clase"""


In [117]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
32287,"Cecilia Moreau: ""No se puede permitir que la minoría le imponga a la mayoría qué temas discutir"" https://t.co/DIOKystxnf",@usuario COCHINA MENTIROSA!! CÓMPLICE canalla de la mafia KAKA!!,"política, aspecto"
26017,Coronavirus en Argentina: el Gobierno convocará a miles de piqueteros para convencer a la gente que se vacune https://t.co/XofDz33SV1,@usuario Los quieren concencer a los palazos? Vacunena los piqueteros y pkanwros primero. @usuario @usuario @usuario @usuario @usuario,"clase, política"
7622,"""Día menstrual para estudiantes "": el proyecto de ley de Carmela Moreau https://t.co/OfxmKpX7bT https://t.co/YxTJbSrQ3k","@usuario Gobierno nacional popular y generador de excusas para no trabajar, y digo gobierno x que esta inepta, incapaz, indigna y descerebrada de Carmela tiene el sello kk ya incorporado en el adn #GobiernoDeInutiles #gobiernodemierda #GobiernoDeMierda","mujer, política"
28206,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,"@usuario hay que hacer guardia, meter bala y enterrar a los soretes para abonar la tierra","política, criminal"
28983,'País de maricas': ¿Por qué Jair Bolsonaro vuelve a atacar a los homosexuales en Brasil? https://t.co/pVdFjiztkm,"@usuario El discurso de bolsonaro donde dice ""maricas"" es brillante, sólo eso basta para meresca ser el presidente de Sudamérica. Clarín es una cueva de zurdos y progresistas que se pintan los labios punto. Clarín suavecito, te pasas de marica!","lgbti, política"
